In [4]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from autogluon.tabular import TabularPredictor

In [5]:
df = pd.read_csv('../../data/data_ze.csv')
df.head(), df.shape

(             gene_id  chromosome  global_position  First_Exon_Start B1 B2 B3  \
 0  ENSG00000260861.6          20          1539145              1000  c  t  g   
 1  ENSG00000260861.6          20          1550939             12794  g  c  c   
 2  ENSG00000260861.6          20          1550961             12816  a  g  g   
 3  ENSG00000171873.8          20          4219631              1000  g  g  t   
 4  ENSG00000101282.9          20           957453              1000  g  a  g   
 
   B4 B5 B6  ... B542 B543 B544 B545 B546 B547 B548 B549 B550 label  
 0  a  a  t  ...    g    a    g    c    a    g    g    c    t  True  
 1  a  a  g  ...    g    t    c    g    t    a    a    g    c  True  
 2  t  c  t  ...    t    g    t    a    a    a    t    c    a  True  
 3  g  c  g  ...    g    a    g    c    c    g    c    c    g  True  
 4  g  a  c  ...    c    g    g    c    g    c    g    c    c  True  
 
 [5 rows x 555 columns],
 (2606, 555))

In [6]:
seq_cols = [col for col in df.columns if col.startswith("B")]
df_model = df[['gene_id'] +seq_cols + ["label"]].copy()

df_model["label"] = (
    df_model["label"].astype(str).str.lower().map({"true": 1, "false": 0})
)
df_model["label"].value_counts(dropna=False)


def split_by_gene(data, test_size=0.2, random_state=42):
    
    splitter = GroupShuffleSplit(test_size=test_size, n_splits=1, random_state=random_state)
    idx_train, idx_test = next(splitter.split(data, groups=data["gene_id"]))

    return data.iloc[idx_train].copy(), data.iloc[idx_test].copy()



train_val, test_data = split_by_gene(df_model, test_size=0.2, random_state=42)
train_data, val_data = split_by_gene(train_val, test_size=0.25, random_state=42)


train = train_data.drop(columns=["gene_id"])
val = val_data.drop(columns=["gene_id"])
test = test_data.drop(columns=["gene_id"]) 

train.shape, val.shape, test_data.shape

((1439, 551), (509, 551), (658, 552))

In [7]:
predictor = TabularPredictor(
    label='label',
    problem_type='binary',
    eval_metric='f1',
    path='../models/autogluon_ze'
)

predictor.fit(
    train_data=train,
    tuning_data=val,
    time_limit=200,
    presets='medium_quality_faster_train'
)


Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025
CPU Count:          12
Pytorch Version:    2.6.0
CUDA Version:       12.6
GPU Memory:         GPU 0: 6.00/6.00 GB
Total GPU Memory:   Free: 6.00 GB, Allocated: 0.00 GB, Total: 6.00 GB
GPU Count:          1
Memory Avail:       6.35 GB / 11.55 GB (55.0%)
Disk Space Avail:   638.50 GB / 951.65 GB (67.1%)
Presets specified: ['medium_quality_faster_train']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 200s
AutoGluon will save models to "/mnt/c/Users/Carlos/OneDrive/Documentos/Genome-Transition_AutoGluon/training/models/autogluon_ze"
Train Data Rows:    1439
Train Data Columns: 550
Tuning Data Rows: 

In [12]:
perf = predictor.evaluate(test,silent=True)
perf

{'f1': 0.6099071207430341,
 'accuracy': 0.6170212765957447,
 'balanced_accuracy': np.float64(0.6406188173755272),
 'mcc': 0.27923975968306297,
 'roc_auc': np.float64(0.7121514809345957),
 'precision': 0.5116883116883116,
 'recall': 0.7547892720306514}

In [9]:
predictor.leaderboard(val, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.683398,0.676409,f1,0.497059,0.371283,78.899404,0.010895,0.001305,0.199532,2,True,12
1,NeuralNetTorch,0.676806,0.676349,f1,0.332315,0.283874,73.477790,0.332315,0.283874,73.477790,1,True,10
2,LightGBMXT,0.662651,0.675159,f1,0.074343,0.141626,2.031774,0.074343,0.141626,2.031774,1,True,1
3,CatBoost,0.659960,0.640000,f1,0.154628,0.092765,40.328495,0.154628,0.092765,40.328495,1,True,5
4,ExtraTreesGini,0.658065,0.491329,f1,0.263431,0.224567,0.802454,0.263431,0.224567,0.802454,1,True,6
5,RandomForestEntr,0.646018,0.500000,f1,0.224096,0.228389,0.802548,0.224096,0.228389,0.802548,1,True,4
6,LightGBMLarge,0.641732,0.649789,f1,0.135535,0.055755,1.516522,0.135535,0.055755,1.516522,1,True,11
7,LightGBM,0.638889,0.639659,f1,0.065901,0.051415,1.286047,0.065901,0.051415,1.286047,1,True,2
8,RandomForestGini,0.638831,0.529412,f1,0.218651,0.120125,0.908132,0.218651,0.120125,0.908132,1,True,3
9,XGBoost,0.627957,0.628571,f1,0.153849,0.086104,5.222081,0.153849,0.086104,5.222081,1,True,9


In [10]:
predictor.feature_importance(
    val,
    subsample_size=min(200, len(val)),
    num_shuffle_sets=3
).head(10)

Computing feature importance via permutation shuffling for 550 features using 200 rows with 3 shuffle sets...
	1318.34s	= Expected runtime (439.45s per shuffle set)
	154.66s	= Actual runtime (Completed 3 of 3 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
B391,0.031624,0.007334,0.008729,3,0.073648,-0.010400
B197,0.029132,0.006103,0.007158,3,0.064104,-0.005840
B182,0.027766,0.010661,0.022895,3,0.088853,-0.033321
B263,0.027109,0.007653,0.012775,3,0.070961,-0.016742
B238,0.026578,0.010346,0.023490,3,0.085861,-0.032706
B59,0.025607,0.001658,0.000697,3,0.035107,0.016107
B341,0.025582,0.006900,0.011701,3,0.065119,-0.013955
B435,0.025304,0.014146,0.045145,3,0.106360,-0.055753
B186,0.025265,0.005045,0.006516,3,0.054172,-0.003643
B112,0.025200,0.008681,0.018675,3,0.074940,-0.024540
